In [2]:
import praw
import os
import json
import time

from supabase import create_client, Client

# Supabase setup
url: str = os.environ.get('SUPABASE_WATCHDB_URL')
key: str = os.environ.get('SUPABASE_WATCHDB_SERVICE_ROLE_KEY')
supabase: Client = create_client(url, key)

# Reddit API Credentials
client_id = os.environ.get('REDDIT_APP_ID')
client_secret = os.environ.get('REDDIT_APP_KEY')
user_agent = 'User-Agent:chrono-codex-server:v1 (by /u/ChronoCrawler)'

# Initialize PRAW with your credentials
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)


Lad wrote a Python script to download Alexa voice recordings, he didn't expect this email.
This post has:
I redesign the Python logo to make it more modern
Automate the boring stuff with python - tinder
Just finished programming and building my own smart mirror in python, wrote all of the code myself and implemented my own voice control and facial recognition features


In [5]:
# The subreddit you want to scrape
subreddit = reddit.subreddit('watchexchange')

# Initialize a list to store data before saving to disk
data_list = []

# Fetch the top posts from the subreddit
top_posts = subreddit.top(time_filter="hour", limit=10)  # Adjust limit as needed


# Push the data collected to Supabase
for post in top_posts:
    post.comments.replace_more(limit=25)  # Load all comments
    # comments = [{'userid': comment.author.name, 'comment': comment.body} for comment in post.comments.list()]
    comments = ' | '.join([f"{comment.author.name}: {comment.body}" for comment in post.comments.list()])

    post_data = {
        'post_id': post.id,
        'author_id': post.author.name,
        'title': post.title,
        'url': post.url,
        'comments': comments
    }
    
    try:
        # Attempt to insert post_data into your Supabase table
        data_insert_response = supabase.table('rqueue').insert(post_data).execute()
    except Exception as e:
        if 'duplicate key value violates unique constraint "rqueue_pkey"' in str(e):
            print(f"Skipping insertion for post_id={post_data['post_id']} as it already exists.")
        else:
            raise


2024-02-06 17:53:19,214:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 409 Conflict"


Skipping insertion for post_id=1akq7lk as it already exists.


2024-02-06 17:53:19,456:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 201 Created"
2024-02-06 17:53:19,743:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 201 Created"
2024-02-06 17:53:20,017:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 201 Created"
2024-02-06 17:53:20,316:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 201 Created"
2024-02-06 17:53:20,715:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 201 Created"
2024-02-06 17:53:20,948:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 201 Created"
2024-02-06 17:53:21,310:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 201 Created"
2024-02-06 17:53:21,584:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v

In [42]:
from jsonschema import validate
from jsonschema.exceptions import ValidationError


def filter_invalid_entries(data, schema):
    """
    Removes entries from the data that do not match the schema's type requirements.
    
    :param data: The JSON object to filter.
    :param schema: The schema defining the expected types.
    :return: A new dictionary with only the valid entries according to the schema.
    """
    valid_data = {}
    for key, value in data.items():
        expected_type = schema.get("properties", {}).get(key, {}).get("type", None)
        if expected_type:
            if expected_type == "number" and value is not None:
                print(key,value)
                try:
                    converted_value = int(value)
                except ValueError:
                    try:
                        converted_value = float(value)
                    except ValueError:
                        continue
                valid_data[key] = converted_value
            elif expected_type == "string" and isinstance(value, str):
                valid_data[key] = value
    return valid_data



def validate_schema(data: dict):
    # This code validates the schema of the provided JSON data
    # and drops any entries that do not match the schema, effectively
    # filtering out incorrect data.

    # Load the schema
    with open('watch_schema.json', 'r') as file:
        schema = json.load(file)

    json_data = filter_invalid_entries(data, schema)

    # Validate the JSON, on failure throw exception
    validate(instance=json_data, schema=schema)
    return json_data  
    


In [44]:
response_json = json.loads(response.choices[0].message.content)
print(response_json)
validated_response = validate_schema(response_json)
validated_response

{'Brand': 'Breitling', 'Model': 'SuperOcean II 36', 'Reference Number': 'A17312', 'Case Material': 'Not specified', 'Case Diameter': None, 'Case Thickness': None, 'Lug Width': None, 'Lug-to-Lug': None, 'Dial Color': 'Not specified', 'Crystal Type': 'Not specified', 'Water Resistance': 'Not specified', 'Movement': 'Not specified', 'Caliber': 'Not specified', 'Movement Type': 'Not specified', 'Power Reserve': 'Not specified', 'Bracelet/Strap Material': 'Not specified', 'Clasp Type': 'Not specified', 'Product Weight': 'Not specified', 'Features': 'Completely full kit with watch roll. 2016 year.', 'Price': 1699, 'Availability': 'Zelle, Venmo, or WU accepted', 'Photo URL': 'https://imgur.com/a/Quq7MB9', 'Merchant Name': 'Pristine_Courage_535', 'Product URL': 'https://www.reddit.com/r/Watchexchange/comments/1akq7lk/wts_breitling_superocean_a17312_white_full_kit/'}
Price 1699


{'Brand': 'Breitling',
 'Model': 'SuperOcean II 36',
 'Reference Number': 'A17312',
 'Case Material': 'Not specified',
 'Dial Color': 'Not specified',
 'Crystal Type': 'Not specified',
 'Water Resistance': 'Not specified',
 'Movement': 'Not specified',
 'Caliber': 'Not specified',
 'Movement Type': 'Not specified',
 'Power Reserve': 'Not specified',
 'Bracelet/Strap Material': 'Not specified',
 'Clasp Type': 'Not specified',
 'Product Weight': 'Not specified',
 'Features': 'Completely full kit with watch roll. 2016 year.',
 'Price': 1699,
 'Availability': 'Zelle, Venmo, or WU accepted',
 'Photo URL': 'https://imgur.com/a/Quq7MB9',
 'Merchant Name': 'Pristine_Courage_535',
 'Product URL': 'https://www.reddit.com/r/Watchexchange/comments/1akq7lk/wts_breitling_superocean_a17312_white_full_kit/'}

In [48]:
from openai import OpenAI
import os
import json

# Set the API key
client = OpenAI(
    api_key= os.environ.get('OPENAI_API_CHRONO_KEY')
)

with open('query_schema.json') as f:
    output_schema_str = f.read()

# Fetch data from Supabase queue
try:
    queue_data = supabase.table('rqueue').select('*').eq('processed', False).limit(1).execute()
    if len(queue_data.data) < 2:  # Fixed to check for non-empty data
        for item in queue_data.data:
            relevant_data = {key: item[key] for key in ["author_id", "title", "url", "comments"]}
            item_json = json.dumps(relevant_data)
            prompt = f"Given the data: {item_json}, construct a JSON object that adheres to the specified output schema. Output schema: {output_schema_str}"
            try:
                response = client.chat.completions.create( 
                    model="gpt-3.5-turbo-0125", 
                    response_format={ "type": "json_object" },
                    messages=[{"role": "system", "content": "You are a helpful assistant that outputs valid JSON.>"}, 
                                {"role": "user", "content": prompt}],
                )
                try:
                    response_json = json.loads(response.choices[0].message.content)
                except Exception as e:
                    print("Error in openai response: ", e)

                try:
                    validated_response = validate_schema(response_json)
                    try:
                        # supabase.table("watches").insert([validated_response]).execute()
                        supabase.table("rqueue").update({"processed": True}).eq("post_id", item["post_id"]).execute()
                    except Exception as e:
                        print(f"Failed to push to supabase (watches): {e}")
                except Exception as e:
                    print(f"current response could not be validated: {e}")
                
            except Exception as e:
                print(f"An OpenAI error occurred: {e}")
        

except Exception as e:
    print(f"Failed to fetch data from Supabase (rqueue): {e}")


2024-02-07 11:37:51,075:INFO - HTTP Request: GET https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue?select=%2A&processed=eq.False&limit=1 "HTTP/1.1 200 OK"
2024-02-07 11:37:51,152:INFO - HTTP Request: PATCH https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue?post_id=eq.1akqglv "HTTP/1.1 200 OK"


<class 'postgrest.base_request_builder.APIResponse'>
data=[{'created_at': '2024-02-07T01:53:19.464226+00:00', 'post_id': '1akqglv', 'author_id': 'liuserr', 'title': '[WTS] Sinn 356 on OEM Leather strap', 'url': 'https://i.redd.it/18ictee7d2hc1.jpeg', 'comments': 'AutoModerator: * u/liuserr has 149 Transactions and [this 7-day post history](https://www.reddit.com/r/Watchexchange/wiki/user_post_history/liuserr).\n* [Click send on this message](https://www.reddit.com/message/compose/?to=WatchExBot&subject=Feedback_Check&message=u/liuserr) to get a detailed look at this user\'s transaction history. \n  * If the above link doesn\'t work, send a message containing the username (starting with u/) to the bot, `WatchExBot`\n* This post: "[[WTS] Sinn 356 on OEM Leather strap](https://www.reddit.com/r/Watchexchange/comments/1akqglv/wts_sinn_356_on_oem_leather_strap/)"  \n\n---\n\n**This post may be removed without notice if it does not follow the rules!**\n\n* **Pictures** are required. Do you ha